In [8]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:10pt;}
div.text_cell_render.rendered_html{font-size:10pt;}
div.output {font-size:10pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:10pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:11pt;padding:4px;}
table.dataframe{font-size:10px;}
</style>
"""))

<b><font size="5" color="red">ch11. 데이터프레임과 시리즈(Pandas)</font></b>
# 9절. 데이터 그룹화 및 집계

In [7]:
from statsmodels.api import datasets
iris = datasets.get_rdataset('iris').data
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


## 9.1 Goupby
cf.pivot_table

In [18]:
# 1) 그룹화 방법 1
#       df.groupby(by=[그룹핑할열])[함수적용해서 출력할 열들].기초통계함수()
#             [함수적용해서 출력할 열들] 생략시 그룹핑할열을  제외한 모든 열 적용
iris_groupby = iris.groupby(iris.Species)
iris_groupby = iris.groupby(['Species'])
iris_groupby = iris.groupby('Species')
for group, data in iris_groupby:
    print(group)
    display(data.sample(5).reset_index())

setosa


,index,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,27,5.2,3.5,1.5,0.2,setosa
1,17,5.1,3.5,1.4,0.3,setosa
2,28,5.2,3.4,1.4,0.2,setosa
3,18,5.7,3.8,1.7,0.3,setosa
4,43,5.0,3.5,1.6,0.6,setosa


versicolor


,index,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,98,5.1,2.5,3.0,1.1,versicolor
1,90,5.5,2.6,4.4,1.2,versicolor
2,59,5.2,2.7,3.9,1.4,versicolor
3,66,5.6,3.0,4.5,1.5,versicolor
4,54,6.5,2.8,4.6,1.5,versicolor


virginica


,index,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,100,6.3,3.3,6.0,2.5,virginica
1,126,6.2,2.8,4.8,1.8,virginica
2,135,7.7,3.0,6.1,2.3,virginica
3,124,6.7,3.3,5.7,2.1,virginica
4,128,6.4,2.8,5.6,2.1,virginica


In [26]:
display(iris.groupby(by='Species').mean()[['Petal.Length', 'Petal.Width']])
display(iris.groupby(by='Species')[['Petal.Length', 'Petal.Width']].mean())

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [22]:
iris_grouped_sum = iris.groupby('Species', as_index=True).median()
# iris_grouped_sum.reset_index()
iris_grouped_sum

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3
virginica,6.5,3.0,5.55,2.0


In [24]:
iris_grouped_sum.loc[['setosa','versicolor']]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
Species,,,,
setosa,5.0,3.4,1.50,0.2
versicolor,5.9,2.8,4.35,1.3


In [21]:
iris_grouped_sum2 = iris.groupby('Species', as_index=False).sum()
iris_grouped_sum2

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3
2,virginica,329.4,148.7,277.6,101.3


In [25]:
iris_grouped_sum2[(iris_grouped_sum2.Species=='setosa') |
                 (iris_grouped_sum2.Species=='versicolor')]

,Species,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,setosa,250.3,171.4,73.1,12.3
1,versicolor,296.8,138.5,213.0,66.3


In [ ]:
# 2) 그룹화 방법 2
#       df.pivot_table(index=[그룹핑할열],
#                      values=[함수적용해서 출력할 열들],
#                      aggfunc=함수(기본값:mean))

#       df.groupby(by=[그룹핑할열])[함수적용해서 출력할 열들].기초통계함수()

In [28]:
iris_groupby('Species')['Petal.Length', 'Petal.Width'].mean()
iris.pivot_table(index='Species',
                values=['Petal.Length','Petal.Width'],
                aggfunc='mean')

,Petal.Length,Petal.Width
Species,,
setosa,1.462,0.246
versicolor,4.260,1.326
virginica,5.552,2.026


In [34]:
iris.groupby('Species').count() # 결측치를 제외한 값의 갯수
iris.pivot_table(index='Species',
                #values=['Sepal.Length','Sepal.Width','Petal.Length','Petal.Width'],
                aggfunc='count')

,Petal.Length,Petal.Width,Sepal.Length,Sepal.Width
Species,,,,
setosa,50,50,50,50
versicolor,50,50,50,50
virginica,50,50,50,50


In [33]:
iris.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


### 다중열로 그룹화

In [35]:
# iris로 다중열로 그룹화하기 위해 데이터 컬럼 추가(num 필드)
iris.shape

(150, 5)

In [40]:
import numpy as np
np.ravel([[i]*25 for i in range(6)])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5])

In [41]:
iris['num'] = np.ravel([[i]*25 for i in range(6)])
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,1
50,7.0,3.2,4.7,1.4,versicolor,2
75,6.6,3.0,4.4,1.4,versicolor,3
100,6.3,3.3,6.0,2.5,virginica,4
125,7.2,3.2,6.0,1.8,virginica,5


In [45]:
# 각열의 임의의 데이터에 결측치를 할당
# iris.loc[0, 0] = np.nan
iris.head(2)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,num
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,0


In [47]:
import random
print(random.sample(range(149), 3))
for col in range(iris.shape[1]-2): #iris.shape[1]-2 : 4
    iris.iloc[random.sample(range(150), col+1), col] = np.nan


[92, 133, 42]


In [48]:
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  149 non-null    float64
 1   Sepal.Width   148 non-null    float64
 2   Petal.Length  147 non-null    float64
 3   Petal.Width   146 non-null    float64
 4   Species       150 non-null    object 
 5   num           150 non-null    int32  
dtypes: float64(4), int32(1), object(1)
memory usage: 6.6+ KB


In [51]:
iris.groupby(by=['Species', 'num'])[['Petal.Length','Petal.Width']].count()

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              24           24
versicolor 2              24           25
           3              25           24
virginica  4              25           25
           5              24           23

In [53]:
iris.pivot_table(index=['Species','num'], values=['Petal.Length','Petal.Width'], aggfunc='count')

Petal.Length  Petal.Width
Species    num                           
setosa     0              25           25
           1              24           24
versicolor 2              24           25
           3              25           24
virginica  4              25           25
           5              24           23

In [56]:
iris.groupby('Species').describe().T

Species                setosa  versicolor  virginica
Sepal.Length count  49.000000   50.000000  50.000000
             mean    5.014286    5.936000   6.588000
             std     0.351188    0.516171   0.635880
             min     4.300000    4.900000   4.900000
             25%     4.800000    5.600000   6.225000
             50%     5.000000    5.900000   6.500000
             75%     5.200000    6.300000   6.900000
             max     5.800000    7.000000   7.900000
Sepal.Width  count  50.000000   50.000000  48.000000
             mean    3.428000    2.770000   2.977083
             std     0.379064    0.313798   0.328271
             min     2.300000    2.000000   2.200000
             25%     3.200000    2.525000   2.800000
             50%     3.400000    2.800000   3.000000
             75%     3.675000    3.000000   3.200000
             max     4.400000    3.400000   3.800000
Petal.Length count  49.000000   49.000000  49.000000
             mean    1.459184    4.255102   5.534694
             std     0.174306    0.473489   0.543734
             min     1.000000    3.000000   4.500000
             25%     1.400000    4.000000   5.100000
             50%     1.500000    4.300000   5.500000
             75%     1.500000    4.600000   5.800000
             max     1.900000    5.100000   6.900000
Petal.Width  count  49.000000   49.000000  48.000000
             mean    0.246939    1.322449   2.041667
             std     0.106266    0.198185   0.267242
             min     0.100000    1.000000   1.400000
             25%     0.200000    1.200000   1.800000
             50%     0.200000    1.300000   2.000000
             75%     0.300000    1.500000   2.300000
             max     0.600000    1.800000   2.500000
num          count  50.000000   50.000000  50.000000
             mean    0.500000    2.500000   4.500000
             std     0.505076    0.505076   0.505076
             min     0.000000    2.000000   4.000000
             25%     0.000000    2.000000   4.000000
             50%     0.500000    2.500000   4.500000
             75%     1.000000    3.000000   5.000000
             max     1.000000    3.000000   5.000000

## 9.2 레이블 인코딩

In [61]:
#iris.drop('num', axis=1, inplace=True)
iris.head(5)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,NaN,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [62]:
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
25,5.0,3.0,1.6,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
75,6.6,3.0,4.4,1.4,versicolor
100,6.3,3.3,6.0,2.5,virginica
125,7.2,3.2,6.0,1.8,virginica


In [64]:
# 레이블 인코딩을 지원하는 클래스 import
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # 객체
iris['target'] = le.fit_transform(iris.Species)
iris.iloc[::25]

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,target
0,5.1,3.5,1.4,0.2,setosa,0
25,5.0,3.0,1.6,0.2,setosa,0
50,7.0,3.2,4.7,1.4,versicolor,1
75,6.6,3.0,4.4,1.4,versicolor,1
100,6.3,3.3,6.0,2.5,virginica,2
125,7.2,3.2,6.0,1.8,virginica,2


# 10절. 데이터 구조 변경
- 와디드 포멧 <-> 롱포멧

- 판다스 공식 문서의 melt :
https://pandas.pydata.org/docs/user_guide/reshaping.html#reshaping-by-melt  

- Tidy Data(7page) :
https://vita.had.co.nz/papers/tidy-data.pdf

In [88]:
from statsmodels.api import datasets
airquality = datasets.get_rdataset('airquality', package='datasets').data
airquality.head()

,Ozone,Solar.R,Wind,Temp,Month,Day
0,41.0,190.0,7.4,67,5,1
1,36.0,118.0,8.0,72,5,2
2,12.0,149.0,12.6,74,5,3
3,18.0,313.0,11.5,62,5,4
4,NaN,NaN,14.3,56,5,5


## 10.1 melt를 이용한 언피벗팅(와이드포멧 -> 롱포멧)

In [70]:
air_melted = airquality.melt(id_vars=['Month','Day'], 
                var_name = '변수명',  # 기본값 : variable
                value_name='값')  #기본값 : value
air_melted.iloc[::5]
air_melted.sort_values(by=['Month','Day'], inplace=True)
air_melted

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0
...,...,...,...,...
610,9,29,Temp,76.0
152,9,30,Ozone,20.0
305,9,30,Solar.R,223.0
458,9,30,Wind,11.5


In [73]:
# melt 연습하기 (부동산 와이드포멧 데이터를 롱포멧)
%ls C:\AI_X\download\shareData\부동산_250213\

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 3664-591E

 C:\AI_X\download\shareData\부동산_250213 디렉터리

2025-04-30  오후 02:50    <DIR>          .
2025-04-30  오후 02:50    <DIR>          ..
2021-01-02  오후 03:44             2,163 전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv
2025-04-30  오후 02:04           419,340 주택도시보증공사_전국 신규 민간아파트 분양가격 동향_20241231.csv
               2개 파일             421,503 바이트
               2개 디렉터리  161,580,847,104 바이트 남음


In [173]:
import pandas as pd
df = pd.read_csv(r'C:\AI_X\download\shareData\부동산_250213\전국 평균 평당 분양가격(2013년 9월부터 2015년 8월까지).csv', encoding='cp949')
df.head(3)

,지역,2013년12월,2014년1월,2014년2월,2014년3월,2014년4월,2014년5월,2014년6월,2014년7월,2014년8월,2014년9월,2014년10월,2014년11월,2014년12월,2015년1월,2015년2월,2015년3월,2015년4월,2015년5월,2015년6월,2015년7월,2015년8월
0,서울,18189,17925,17925,18016,18098,19446,18867,18742,19274,19404,19759,20242,20269,20670,20670,19415,18842,18367,18374,18152,18443
1,부산,8111,8111,9078,8965,9402,9501,9453,9457,9411,9258,9110,9208,9208,9204,9235,9279,9327,9345,9515,9559,9581
2,대구,8080,8080,8077,8101,8267,8274,8360,8360,8370,8449,8403,8439,8253,8327,8416,8441,8446,8568,8542,8542,8795


In [174]:
new_df = df.melt(id_vars='지역',
       var_name='연도월',
       value_name='평당분향가')
new_df.sample()

,지역,연도월,평당분향가
255,서울,2015년3월,19415


## 10.2 pivot_table을 이용한 피벗팅(롱포멧 ->와이드포멧)

In [81]:
# air)melted를 와이드 포멧으로
air_melted.head()

,Month,Day,변수명,값
0,5,1,Ozone,41.0
153,5,1,Solar.R,190.0
306,5,1,Wind,7.4
459,5,1,Temp,67.0
1,5,2,Ozone,36.0


In [84]:
airquality2 = air_melted.pivot_table(index=['Month','Day'], columns='변수명', values='값')
airquality2.reset_index(inplace=True)
airquality2.columns.name = None

In [86]:
airquality2.head()

,Month,Day,Ozone,Solar.R,Temp,Wind
0,5,1,41.0,190.0,67.0,7.4
1,5,2,36.0,118.0,72.0,8.0
2,5,3,12.0,149.0,74.0,12.6
3,5,4,18.0,313.0,62.0,11.5
4,5,5,NaN,NaN,56.0,14.3


# 11절. 데이터프레임(시리즈)에 함수적용시키기
- apply(func, axis) : 축요소별 (요소별) 함수적용(데이터프레임이나 시리즈에서 함수 적용)
- applymap(func) : 각 요소별 함수 적용(데이터프레임에서만 가능)
- map(func, list) : 각 요소별 함수 적용(1차원 리스트, 시리즈에서만 가능)
## 11.1 apply

In [93]:
member = pd.read_csv('data/ch11_member.csv', comment='#')
member

,Name,Age,Email,Address,Birth
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30


In [94]:
member['Birth']

0    [1999, 01, 01]
1    [1999, 12, 01]
2    [1999, 11, 01]
3    [1999, 05, 01]
4    [1999, 01, 30]
Name: Birth, dtype: object

In [97]:
def get_year(datestr):
    return int(datestr.spilt('-')[0])

In [116]:
get_year = lambda datestr : int(datestr.split('-')[0])

In [117]:
get_year('1999-05-01')

1999

In [115]:
member['year'] = member['Birth'].apply(lambda datestr : int(datestr.split('-')[0]))
member.head()

,Name,Age,Email,Address,Birth,year
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999


In [122]:
X = iris.iloc[:,:-2]
# x의 열별 평균
avg = X.mean(axis=0)
avg

Sepal.Length    5.851678
Sepal.Width     3.059459
Petal.Length    3.749660
Petal.Width     1.197945
dtype: float64

In [123]:
X.iloc[0]-avg

Sepal.Length   -0.751678
Sepal.Width     0.440541
Petal.Length   -2.349660
Petal.Width    -0.997945
dtype: float64

In [124]:
# X의 모든행에 대하여 열별 평균과의 거리를 소수점 2자리까지 출력
X.apply(lambda x : round(x-avg, 2), axis=1)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,-0.75,0.44,-2.35,-1.0
1,-0.95,-0.06,-2.35,-1.0
2,-1.15,0.14,-2.45,-1.0
3,NaN,0.04,-2.25,-1.0
4,-0.85,0.54,-2.35,-1.0
...,...,...,...,...
145,0.85,-0.06,1.45,1.1
146,0.45,-0.56,1.25,0.7
147,0.65,-0.06,1.45,0.8
148,0.35,0.34,1.65,1.1


## 11.2 applymap

In [125]:
X.applymap(np.round)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
0,5.0,4.0,1.0,0.0
1,5.0,3.0,1.0,0.0
2,5.0,3.0,1.0,0.0
3,NaN,3.0,2.0,0.0
4,5.0,4.0,1.0,0.0
...,...,...,...,...
145,7.0,3.0,5.0,2.0
146,6.0,2.0,5.0,2.0
147,6.0,3.0,5.0,2.0
148,6.0,3.0,5.0,2.0


## 11.3 map
- 1차원

In [129]:
member['Month'] = member['Birth'].map(lambda datestr : int(datestr.split('-')[1]))
member

,Name,Age,Email,Address,Birth,year,Month
0,홍길동,20,hong@hong.com,서울시 강동구,1999-01-01,1999,1
1,홍길서,30,seo@hong.com,서울시 중구,1999-12-01,1999,12
2,홍길남,21,nam@hong.com,서울시 강동구,1999-11-01,1999,11
3,홍길북,22,buk@hong.com,서울시 관악구,1999-05-01,1999,5
4,홍길중,23,mid@hong.com,부산시 중구,1999-01-30,1999,1


In [130]:
# 연습하기(apply, map)
# new_df의 연도월 데이터를 연도와 월을 int로 분리하여 컬렴으로 추가

,지역,연도월,평당분향가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098
...,...,...,...
352,전북,2015년8월,6580
353,전남,2015년8월,6289
354,경북,2015년8월,7037
355,경남,2015년8월,7665


In [148]:
new_df['년도'] = new_df['연도월'].apply(lambda str : int(str.split('년')[0]))
new_df['월'] = new_df['연도월'].apply(lambda str : int(str.split('년')[1][:-1]))
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 357 entries, 0 to 356
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      357 non-null    object
 1   연도월     357 non-null    object
 2   평당분향가   357 non-null    int64 
 3   년도      357 non-null    int64 
 4   월       357 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 14.1+ KB


In [179]:
new_df.sample()

,지역,연도월,평당분향가,연도,월
13,전남,2013년12월,5678,2013,12


In [180]:
new_df.drop(['연도','월'], axis=1, inplace=True)
new_df.head()

,지역,연도월,평당분향가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098


In [ ]:
# ★ 2차원 데이터 프레임의 apply 이용하는 방법

In [185]:
new_df

,지역,연도월,평당분향가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098
...,...,...,...
352,전북,2015년8월,6580
353,전남,2015년8월,6289
354,경북,2015년8월,7037
355,경남,2015년8월,7665


In [175]:
def split_date(df):
    df = df.copy()
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1][:-1])
    return df

In [176]:
split_date(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분향가       18189
연도           2013
월              12
Name: 0, dtype: object

In [177]:
new_df = new_df.apply(split_date, axis=1)
new_df.sample(

,지역,연도월,평당분향가,연도,월
6,울산,2013년12월,8090,2013,12


In [196]:
new_df

,지역,연도월,평당분향가
0,서울,2013년12월,18189
1,부산,2013년12월,8111
2,대구,2013년12월,8080
3,인천,2013년12월,10204
4,광주,2013년12월,6098
...,...,...,...
352,전북,2015년8월,6580
353,전남,2015년8월,6289
354,경북,2015년8월,7037
355,경남,2015년8월,7665


In [199]:
def year_month_append(df):
    'df를 받아 연도와 월 추가후 reutrn'
    df = df.copy() # 깊은 복사
    df['연도'] = int(df['연도월'].split('년')[0])
    df['월'] = int(df['연도월'].split('년')[1].replace('월',''))
    return df

In [200]:
year_month_append(new_df.iloc[0])

지역             서울
연도월      2013년12월
평당분향가       18189
연도           2013
월              12
Name: 0, dtype: object

In [201]:
new_df = new_df.apply(year_month_append, axis=1)
new_df.head()

,지역,연도월,평당분향가,연도,월
0,서울,2013년12월,18189,2013,12
1,부산,2013년12월,8111,2013,12
2,대구,2013년12월,8080,2013,12
3,인천,2013년12월,10204,2013,12
4,광주,2013년12월,6098,2013,12


# 12절. 일괄변경(결측치, 특정값)
## 12.1 fillna
- 대부분 결측치 drop하거나, 평균값대체, 중위수대체, 그룹별 평균값, 그룹별 중위수, 이전값, 이후값 등으로 대체

In [187]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0],
                   [3,      4, np.nan, 1],
                   [np.nan, 3, np.nan, 5],
                   [np.nan, 3, np.nan, 4]],
                 columns=[list('ABCD')])
df

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [188]:
# 모든 결측치를 0으로 대체
df.fillna(0)

,A,B,C,D
0,0.0,2,0.0,0
1,3.0,4,0.0,1
2,0.0,3,0.0,5
3,0.0,3,0.0,4


In [189]:
# 결측치를 이전행의 값으로 대체
df.fillna(method='ffill')

,A,B,C,D
0,NaN,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [191]:
# 결측치를 다음행의 값으로 대체
df.fillna(method='bfill')

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,NaN,3,NaN,5
3,NaN,3,NaN,4


In [192]:
# 결측치를 열마다 특정값으로 대체
values = {'A':85, 'B':80,'C':75,'D':100}
df.fillna(value=values)

,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,75.0,5
3,85.0,3,75.0,4


In [194]:
# 결측치를 열별 평균값으로 대체
df.mean(axis=0)
df.fillna(value=df.mean(axis=0))

,A,B,C,D
0,3.0,2,NaN,0
1,3.0,4,NaN,1
2,3.0,3,NaN,5
3,3.0,3,NaN,4


In [195]:
# 결측치 대체를 2번째 결측치까지만 대체
print(values)
df.fillna(value=values, limit=2)

{'A': 85, 'B': 80, 'C': 75, 'D': 100}


,A,B,C,D
0,85.0,2,75.0,0
1,3.0,4,75.0,1
2,85.0,3,NaN,5
3,NaN,3,NaN,4


## 12.2 replace(to_value, new_value, inplace=T/F)
- to_value를 new_value
- to_value에 정규표현식 이용